In [1]:
import json
import os
import string

In [2]:
def loadmsgs(fn):
    if os.path.getsize(fn) <= 0:
        return [], []
    
    lstbg = []
    lstfg = []
    
    with open(fn, "r") as read_file:
        msgs = json.load(read_file)
        curfg = []
        isfg = False
        
        for v in msgs:
            co = json.loads(v)
            
            if 'allowedGameFeatures' not in co and 'error' not in co and 'game' in co:
                if co['game']['mode'] == 'NORMAL' and co['game']['nextMode'] == 'NORMAL':
                    lstbg.append(co)
                
                if co['game']['mode'] == 'NORMAL' and co['game']['nextMode'] == 'FREESPIN':
                    curfg = []
                    curfg.append(co)
                    isfg = True
                
                if isfg and co['game']['mode'] == 'FREESPIN':
                    curfg.append(co)
                    if co['game']['nextMode'] == 'NORMAL':
                        isfg = False
                        lstfg.append(curfg)
                
    
    return lstbg, lstfg

def loadAllFiles():
    lstbg = []
    lstfg = []
    
    dirs = os.listdir('./')
    for v in dirs:
        if v.endswith('.json'):
            clstbg, clstfg = loadmsgs(v)
            lstbg = lstbg + clstbg
            lstfg = lstfg + clstfg
    
    return lstbg, lstfg

def analyzeWins(lstbg, lstfg):
    wins = {}
    
    for v in lstbg:
        if 'realityCheck' in v:
            k = str(v['realityCheck']['winnings'])
            if k in wins:
                wins[k] = wins[k] + 1
            else:
                wins[k] = 1
                
    for arr in lstfg:
        curwin = 0
        
        for v in arr:
            if 'realityCheck' in v:
                curwin = curwin + v['realityCheck']['winnings']
            
        k = str(curwin)
        if k in wins:
            wins[k] = wins[k] + 1
        else:
            wins[k] = 1
    
    return wins

def analyzeReels(lstbg, lstfg, reelnum):
    bgreels = []
    fgreels = []    
    for i in range(reelnum):
        bgreels.append({})
        fgreels.append({})        
    
    for v in lstbg:
        vg = v['game']
        if 'reels' in vg:
            for i in range(len(vg['reels'])):
                reel = vg['reels'][i]

                for s in reel:
                    if s in bgreels[i]:
                        bgreels[i][s] = bgreels[i][s] + 1
                    else:
                        bgreels[i][s] = 1
                            
    for arr in lstfg:
        for i in range(len(arr)):
            vg = arr[i]['game']            
            
            if i == 0:
                if 'reels' in vg:
                    for i in range(len(vg['reels'])):
                        reel = vg['reels'][i]

                        for s in reel:
                            if s in bgreels[i]:
                                bgreels[i][s] = bgreels[i][s] + 1
                            else:
                                bgreels[i][s] = 1
            else:
                if 'reels' in vg:
                    for i in range(len(vg['reels'])):
                        reel = vg['reels'][i]

                        for s in reel:
                            if s in fgreels[i]:
                                fgreels[i][s] = fgreels[i][s] + 1
                            else:
                                fgreels[i][s] = 1                                         
    
    return bgreels, fgreels

def analyzeSymbolWins(lst):
    symbolWins = {}
    
    for v in lst:
        if 'game' in v:
            vg = v['game']
            if 'betWayWins' in vg:
                for win in vg['betWayWins']:
                    if win['symbol'] in symbolWins:
                        if win['length'] in symbolWins[win['symbol']]:
                            symbolWins[win['symbol']][win['length']] = symbolWins[win['symbol']][win['length']] + 1
                        else:
                            symbolWins[win['symbol']][win['length']] = 1
                    else:
                        symbolWins[win['symbol']] = {}
                        symbolWins[win['symbol']][win['length']] = 1
    
    return symbolWins

def findFG(lst):
    lstfg = []
    
    for v in lst:
        if v['game']['fslevel'] != 0:
            lstfg.append(v)
    
    return lstfg

def findBG(lst):
    lstbg = []
    
    for v in lst:
        if v['game']['mode'] == 'NORMAL' and v['game']['nextMode'] == 'NORMAL':
            lstbg.append(v)
    
    return lstbg

def getAllReels(lst):
    lstreels = {}
    
    for v in lst:
        if v['game']['reelsNextSpin'] in lstreels:
            lstreels[v['game']['reelsNextSpin']] = lstreels[v['game']['reelsNextSpin']] + 1
        else:
            lstreels[v['game']['reelsNextSpin']] = 1
    
    return lstreels

def getAllModes(lst):
    lstmodes = {}
    
    for v in lst:
        if v['game']['mode'] in lstmodes:
            lstmodes[v['game']['mode']] = lstmodes[v['game']['mode']] + 1
        else:
            lstmodes[v['game']['mode']] = 1
            
        if v['game']['nextMode'] in lstmodes:
            lstmodes[v['game']['nextMode']] = lstmodes[v['game']['nextMode']] + 1
        else:
            lstmodes[v['game']['nextMode']] = 1            
    
    return lstmodes

def findMysterySymbolReveal(lst):
    lstReveal = {}
    
    for v in lst:
        if v['game']['mysterySymbolReveal'] in lstReveal:
            lstReveal[v['game']['mysterySymbolReveal']] = lstReveal[v['game']['mysterySymbolReveal']] + 1
        else:
            lstReveal[v['game']['mysterySymbolReveal']] = 1
    
    return lstReveal

def findMystery(lst):
    lstm = []
    
    for v in lst:
        if v['game']['mysterySymbolReveal'] != '':
            lstm.append(v)
    
    return lstm

In [3]:
lst, lstfg = loadAllFiles()

len(lst)

1343

In [4]:
len(lstfg)

7

In [5]:
lstfg[1]

[{'balance': '9958.50',
  'balances': {'TOTAL_BALANCE': '9958.50'},
  'betLevels': {'bet': '1.00',
   'availableBetLevels': ['1.00'],
   'unavailableBetLevels': [],
   'message': ''},
  'extPlayerKey': 'a4f77fac3f3d193949141364e0099509274b74f2af8cf6d5950bd5f9fa2b83f3b0',
  'featureBetLevels': {'FREESPIN_LEVEL_TWO': ['0.20',
    '0.40',
    '0.60',
    '0.80',
    '1.00',
    '1.20',
    '1.60',
    '2.00',
    '2.40',
    '2.80',
    '3.20',
    '3.60',
    '4.00',
    '5.00',
    '6.00',
    '8.00',
    '10',
    '14',
    '18',
    '24',
    '32',
    '40'],
   'FREESPIN': ['0.20',
    '0.40',
    '0.60',
    '0.80',
    '1.00',
    '1.20',
    '1.60',
    '2.00',
    '2.40',
    '2.80',
    '3.20',
    '3.60',
    '4.00',
    '5.00',
    '6.00',
    '8.00',
    '10',
    '14',
    '18',
    '24',
    '32',
    '40',
    '60',
    '80',
    '100'],
   'FREESPIN_LEVEL_ONE': ['0.20',
    '0.40',
    '0.60',
    '0.80',
    '1.00',
    '1.20',
    '1.60',
    '2.00',
    '2.40',
    '2.

In [6]:
wins = analyzeWins(lst, lstfg)

wins

{'0': 979,
 '0.5': 58,
 '0.75': 11,
 '7.5': 1,
 '17.25': 1,
 '0.25': 29,
 '0.8': 14,
 '4.5': 3,
 '48': 1,
 '12': 2,
 '0.2': 85,
 '1.5': 8,
 '0.4': 13,
 '17.6': 1,
 '0.3': 19,
 '4.8': 1,
 '8.8': 1,
 '0.6': 14,
 '1.8': 2,
 '1': 14,
 '2.85': 1,
 '2.6': 1,
 '3': 8,
 '2': 5,
 '1.2': 2,
 '1.1': 8,
 '6': 3,
 '2.4': 4,
 '1.6': 2,
 '4': 1,
 '2.95': 1,
 '2.2': 3,
 '0.55': 4,
 '0.7': 4,
 '2.1': 1,
 '80': 1,
 '1.4': 2,
 '11.25': 1,
 '9.9': 1,
 '2.3': 2,
 '32': 1,
 '27.2': 1,
 '1.7': 1,
 '3.3': 2,
 '3.75': 1,
 '3.2': 1,
 '1.3': 1,
 '0.9': 3,
 '18': 1,
 '2.25': 4,
 '4.7': 1,
 '8.4': 1,
 '4.2': 1,
 '1.25': 2,
 '4.4': 3,
 '1.85': 1,
 '8': 1,
 '54': 1,
 '10': 1,
 '20': 1,
 '2.8': 1,
 '16': 1,
 '64': 1,
 '46.150000000000006': 1,
 '101.25': 1,
 '29.45': 1,
 '341.2': 1,
 '240.9': 1,
 '165.70000000000002': 1}

In [7]:
bgreels, fgreels = analyzeReels(lst, lstfg, 5)

bgreels

[{'L1': 1049,
  'L3': 754,
  'L5': 598,
  'M5': 823,
  'L2': 1111,
  'M1': 105,
  'M3': 335,
  'M4': 780,
  'M2': 441,
  'L4': 754},
 {'L1': 1237,
  'S': 228,
  'M3': 1124,
  'L3': 1094,
  'L5': 1167,
  'M2': 150,
  'R1': 313,
  'M5': 903,
  'M4': 224,
  'M1': 102,
  'L4': 172,
  'L2': 36},
 {'M1': 169,
  'L5': 568,
  'L2': 874,
  'M2': 1116,
  'W': 186,
  'M4': 683,
  'L4': 727,
  'R1': 293,
  'M3': 781,
  'L3': 616,
  'M5': 380,
  'S': 155,
  'L1': 202},
 {'M4': 1199,
  'L5': 550,
  'M5': 214,
  'L4': 1282,
  'L2': 1051,
  'M1': 140,
  'L3': 117,
  'L1': 338,
  'M2': 1113,
  'M3': 186,
  'S': 159,
  'R1': 290,
  'W': 111},
 {'M3': 717,
  'M5': 635,
  'L3': 995,
  'L4': 634,
  'W': 46,
  'L1': 768,
  'M4': 515,
  'L2': 798,
  'L5': 693,
  'M1': 232,
  'M2': 458,
  'S': 259}]

In [15]:
fgreels

[{'L4': 36,
  'L1': 70,
  'L2': 52,
  'L3': 71,
  'M4': 42,
  'M1': 14,
  'M2': 17,
  'M3': 20,
  'L5': 14,
  'M5': 14},
 {'L3': 50,
  'L1': 36,
  'L4': 44,
  'L2': 48,
  'M3': 41,
  'M4': 42,
  'M1': 22,
  'M2': 33,
  'L5': 9,
  'M5': 10,
  'R1': 15},
 {'L1': 54,
  'M4': 37,
  'L4': 43,
  'L2': 48,
  'L3': 48,
  'M2': 30,
  'M1': 30,
  'M3': 29,
  'R1': 10,
  'M5': 8,
  'L5': 13},
 {'L1': 55,
  'M3': 31,
  'M1': 23,
  'L2': 55,
  'L4': 43,
  'L3': 43,
  'R1': 14,
  'M2': 32,
  'M4': 33,
  'M5': 8,
  'L5': 13},
 {'M4': 47,
  'M3': 34,
  'L2': 57,
  'L4': 30,
  'L3': 42,
  'L1': 43,
  'M2': 44,
  'M1': 36,
  'M5': 11,
  'L5': 6}]

In [8]:
symbolwin = analyzeSymbolWins(lst)

symbolwin

{'L1': {3: 65, 4: 13},
 'L2': {3: 8, 5: 2, 4: 6},
 'M2': {3: 7, 4: 4, 5: 2},
 'M3': {4: 4, 3: 42, 5: 2},
 'M5': {3: 43, 5: 2, 4: 8},
 'L5': {3: 48, 4: 20, 5: 6},
 'M4': {5: 4, 4: 7, 3: 12},
 'L3': {3: 72, 4: 5, 5: 5},
 'L4': {4: 7, 3: 6, 5: 1}}

In [9]:
# lstfg = findFG(lst)

# lstfg[0]

In [10]:
# lstfg[8]

In [11]:
lstreels = getAllReels(lst)

lstreels

{'BASE_REELSET': 1343}

In [12]:
lstmodes = getAllModes(lst)

lstmodes

{'NORMAL': 2686}

In [13]:
lstreveal = findMysterySymbolReveal(lst)

lstreveal

{'': 887,
 'L4': 68,
 'L2': 75,
 'L1': 58,
 'L3': 56,
 'M3': 33,
 'M2': 15,
 'M4': 34,
 'L5': 58,
 'M5': 53,
 'M1': 6}

In [14]:
lstm = findMystery(lst)

lstm[0]

{'balance': '10025.85',
 'balances': {'TOTAL_BALANCE': '10025.85'},
 'betLevels': {'bet': '1.00',
  'availableBetLevels': ['0.20',
   '0.40',
   '0.60',
   '0.80',
   '1.00',
   '1.20',
   '1.60',
   '2.00',
   '2.40',
   '2.80',
   '3.20',
   '3.60',
   '4.00',
   '5.00',
   '6.00',
   '8.00',
   '10',
   '14',
   '18',
   '24',
   '32',
   '40',
   '60',
   '80',
   '100'],
  'unavailableBetLevels': [],
  'message': ''},
 'extPlayerKey': 'a4f77fac3f3d193949141360b25f955b751b2ff6ab8ff5d6c05a86acf72ad2f3b0',
 'featureBetLevels': {'FREESPIN_LEVEL_TWO': ['0.20',
   '0.40',
   '0.60',
   '0.80',
   '1.00',
   '1.20',
   '1.60',
   '2.00',
   '2.40',
   '2.80',
   '3.20',
   '3.60',
   '4.00',
   '5.00',
   '6.00',
   '8.00',
   '10',
   '14',
   '18',
   '24',
   '32',
   '40'],
  'FREESPIN': ['0.20',
   '0.40',
   '0.60',
   '0.80',
   '1.00',
   '1.20',
   '1.60',
   '2.00',
   '2.40',
   '2.80',
   '3.20',
   '3.60',
   '4.00',
   '5.00',
   '6.00',
   '8.00',
   '10',
   '14',
   '18'